In [1]:
# Haversine distance formula
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

from w210_attribute_library import haversine_distance

datdir = "../data/"
attrs = "../attrs/"

## Read Weather Data

In [2]:
fweather = 'weather_data.csv'
dfw = pd.read_csv(datdir+fweather)
print(len(dfw))
dfw.head(2)

291511


,stn_wban,year,mo,da,min,max,temp,prcp,flag_prcp,rain_drizzle,...,usaf_wban,wban,lon,lat,elev,name,begin,end,state,country
0,998199_99999,2016,8,12,77.9,84.9,81.5,0.0,I,0,...,998199_99999,99999,-85.667,30.15,5.0,PANAMA CITY,20080721,20210920,FL,US
1,998199_99999,2016,11,22,47.8,66.6,57.2,0.0,I,0,...,998199_99999,99999,-85.667,30.15,5.0,PANAMA CITY,20080721,20210920,FL,US


In [3]:
pivot = np.round(pd.pivot_table(dfw, values='prcp', 
                                index='year', 
                                columns='mo', 
                                aggfunc='count'),2)
pivot

mo,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2015,3541.0,3186.0,3500.0,3366.0,3450.0,3331.0,3403.0,3357.0,3207.0,3306.0,3208.0,3292.0
2016,3274.0,3131.0,3377.0,3251.0,3309.0,3221.0,3333.0,3363.0,3211.0,3288.0,3205.0,3376.0
2017,3333.0,3098.0,3380.0,3264.0,3473.0,3337.0,3412.0,3356.0,3146.0,3321.0,3136.0,3212.0
2018,3484.0,3151.0,3449.0,3260.0,2619.0,2492.0,2582.0,2624.0,2503.0,2525.0,2454.0,2543.0
2019,3453.0,3113.0,3419.0,3301.0,3419.0,3255.0,3452.0,3450.0,3343.0,3408.0,3321.0,3440.0
2020,3409.0,3177.0,3346.0,3248.0,3372.0,3268.0,3380.0,3356.0,3187.0,3350.0,3245.0,3376.0
2021,3312.0,2882.0,3297.0,3346.0,3417.0,3303.0,3380.0,3367.0,3236.0,3339.0,3290.0,3387.0
2022,3422.0,3121.0,3456.0,3351.0,3404.0,2473.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
wsnames = set(dfw["stn_wban"].unique())
len(wsnames)

126

In [5]:
ws = dfw[['stn_wban', "name", "lon", "lat"]].drop_duplicates()
ws.columns = ["Key", 'name', 'lon_w', 'lat_w']
len(ws)

126

In [6]:
duplicateName = ['WHITING FIELD NAVAL AIR STATI', 'KEYSTONE AIRPARK']
ws[(ws['name'].isin(duplicateName))]

,Key,name,lon_w,lat_w
58,749048_99999,KEYSTONE AIRPARK,-82.048,29.845
19492,720383_53847,WHITING FIELD NAVAL AIR STATI,-87.023,30.704
166418,722226_93841,WHITING FIELD NAVAL AIR STATI,-87.017,30.717
219704,749048_00415,KEYSTONE AIRPARK,-82.048,29.845


## Read Tile Data - for coordinates

In [7]:
dftiles = pd.read_excel(attrs+"model_sh_attr_365_1.xlsx")
dtil = dftiles[["Key", "X", "Y", "DateD"]]
dtil.columns = ["Key", 'lon_t', 'lat_t', "DateD"]
print(len(dtil))
dtil.head(1)

276


,Key,lon_t,lat_t,DateD
0,428_1_0,-81.932475,28.944928,2019-12-11


## Take Out Weather Stations without Data
Step implemented when quality check indicates not-enough data available (done at the end of first phase)

List of no-good weather stations in `wsnogood`

In [59]:
len(wsnames)

126

In [61]:
wsgoodList = wsnames - set(wng1)
len(wsgoodList)

119

In [62]:
ws = ws[(ws["Key"].isin(wsgoodList))]
len(ws)

119

## Merge Tile and Weather for Distance Calculation

`result = pd.merge(left, right, how="left", on=["key1", "key2"])`

**Reference:**
https://pandas.pydata.org/docs/user_guide/merging.html

In [63]:
dffinal = pd.merge(dtil,ws, how="cross")
print(len(dffinal))
dffinal.head(1)

32844


,Key_x,lon_t,lat_t,DateD,Key_y,name,lon_w,lat_w
0,428_1_0,-81.932475,28.944928,2019-12-11,999999_63890,WHITEHOUSE NAVAL OUTLYING FIE,-81.883,30.35


## Calculate The Distance

In [64]:
dffinal['Distance'] = dffinal.apply(lambda row: 
                                    haversine_distance(row['lat_t'], row['lon_t'], 
                                                       row['lat_w'], row['lon_w'], earth_radius=3963.19), axis=1)

In [65]:
# dffinal['Key'] = dffinal.apply(lambda row: row['Key_x']+"-"+row['Key_y'], axis=1)
# dffinal.head(1)

## Find the Minimum Distances within Tile and Weather Station

`df.groupby('Company')['MPG'].agg('min')`  
`df.groupby('Company')[['MPG', 'EngineSize']].agg('min')`

**Reference:**
https://datascienceparichay.com/article/pandas-groupby-minimum/

In [66]:
dfmin1 = dffinal.groupby(['Key_x'])['Distance'].min().to_frame()

In [67]:
len(dfmin1)

276

In [68]:
# Test to Find Tile and Closest Weather Station
d = dfmin1['Distance'][0]
k = dfmin1.index[0]
print(d,k)
dffinal[((dffinal['Key_x']==k) &  (dffinal['Distance']==d))]

4.429513905621283 1060_0_1


,Key_x,lon_t,lat_t,DateD,Key_y,name,lon_w,lat_w,Distance
14942,1060_0_1,-87.214351,30.537544,2019-12-18,722223_13899,PENSACOLA REGIONAL AIRPORT,-87.187,30.478,4.429514


In [69]:
keysL = list(dfmin1.index)
minD = list(dfmin1['Distance'])
dfF1 = dffinal[((dffinal['Key_x'].isin(keysL)) &  (dffinal['Distance'].isin(minD)))]
len(dfF1)

276

**Reference Duplicates:**
https://sparkbyexamples.com/pandas/pandas-get-list-of-all-duplicate-rows/#:~:text=Pandas%20DataFrame.,multiple%20columns%20or%20all%20columns.

In [70]:
#Checking for Duplicates
df2 = dfF1[dfF1["Key_x"].duplicated()==True]
dup1 = df2["Key_x"].unique()
# dup1 = ['1082_0_1','1083_0_1', '2406_0_1', '2459_0_1', '2463_0_1', '2737_0_1', '3294_0_2', '3294_1_0', '556_0_1']
dfF1[(dfF1['Key_x'].isin(dup1))]

,Key_x,lon_t,lat_t,DateD,Key_y,name,lon_w,lat_w,Distance


In [71]:
dfF1.drop_duplicates(subset=['Key_x'], inplace=True)
len(dfF1)

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_9099/202467481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfF1.drop_duplicates(subset=['Key_x'], inplace=True)


276

In [72]:
dfF1["Distance"].describe()

count     276.000000
mean       30.537432
std       293.374519
min         0.546547
25%         5.310610
50%        10.355607
75%        17.938391
max      4883.222790
Name: Distance, dtype: float64

In [73]:
tilesIssues = ['2321_1_0', '2734_1_0', '2734_0_2', '2321_0_2']
keysL = set(keysL)
tIssues = set(tilesIssues)
keysFilter = keysL - tIssues
len(keysFilter)

272

In [74]:
# Drop Observations with Distance greater than 150
dfF2 = dfF1[(dfF1["Distance"] < 150)]
len(dfF2)

275

In [75]:
dfF2.to_excel(datdir+"model_tile_ws_clean.xlsx", index=False, sheet_name="tile_ws_pair")

In [76]:
dfF2.head(1)

,Key_x,lon_t,lat_t,DateD,Key_y,name,lon_w,lat_w,Distance
65,428_1_0,-81.932475,28.944928,2019-12-11,722213_12819,LEESBURG REGIONAL AIRPORT,-81.81,28.821,11.336137


## Check Quality of Weather Station

In [77]:
colweather = ['stn_wban', 'year', 'mo', 'da', 'temp', 'prcp', 'flag_prcp']
dfw1 = dfw[colweather]
dfw1.head(1)

,stn_wban,year,mo,da,temp,prcp,flag_prcp
0,998199_99999,2016,8,12,81.5,0.0,I


In [78]:
dfw1["DateD"] = dfw1.apply(lambda row: datetime.strptime(str(row["year"])+"-"+str(row["mo"])+"-"+str(row["da"]),"%Y-%m-%d") ,axis=1)
dfw1.head()

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_9099/2759727536.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfw1["DateD"] = dfw1.apply(lambda row: datetime.strptime(str(row["year"])+"-"+str(row["mo"])+"-"+str(row["da"]),"%Y-%m-%d") ,axis=1)


,stn_wban,year,mo,da,temp,prcp,flag_prcp,DateD
0,998199_99999,2016,8,12,81.5,0.0,I,2016-08-12
1,998199_99999,2016,11,22,57.2,0.0,I,2016-11-22
2,998199_99999,2016,7,28,84.8,0.0,I,2016-07-28
3,998199_99999,2016,4,29,75.7,0.0,I,2016-04-29
4,998199_99999,2016,2,25,52.4,0.0,I,2016-02-25


In [79]:
# dfF2["DateD"] = dfF2.apply(lambda row: datetime.strptime(row["DateD"],"%Y-%m-%d"), axis=1)

## Get List of No-Good Weather Stations

In [80]:
i = 0
wsgood = []
wsnogood = []
for index, row in dfF2.iterrows():
    wsname = row["Key_y"]
    dated = row["DateD"]
    td = timedelta(365*3)
    dftemp = dfw1[(dfw1["stn_wban"] == wsname)]
    if (dftemp["DateD"].max() > dated) & (dftemp["DateD"].min() < dated - td):
        wsgood.append(wsname)
    else:
        wsnogood.append(wsname)

In [81]:
print(len(wsnogood))
wng = set(wsnogood)
list(wng)

0


[]

`
{'722012_92817',
 '722224_53862',
 '747761_99999',
 '749045_99999',
 '998199_99999',
 '999999_53848'}
`

In [55]:
len(wsgood)

274

In [58]:
wng1 = ['747960_99999', '722012_92817',
        '722224_53862','747761_99999',
        '749045_99999','998199_99999',
        '999999_53848']

Final List of Weather Stations without Enough Data:
`
['747960_99999', '722012_92817',
        '722224_53862','747761_99999',
        '749045_99999','998199_99999',
        '999999_53848']
`

### Finish Process Once wsnogood is equal to zero